In [181]:
import sqlite3
from sqlite3 import Error
import pandas as pd
from Database import PySQLite
from CorporateCredit import CorporateCredit
import math
import blpapi
import pdblp
from datetime import date

Today=date.today()
Today_f=str(Today.year)+str(Today.month)+str(Today.day)
pd.options.display.float_format = '{:,.2f}'.format

bbgdict={'1yrchange':'PR070','1mchange':'RQ543'}

# 'PR070' = bbg code for %1yr change (w/o dividend)
# 'RQ543' = bbg code for %1yr change (w/o dividend)


class CompareCredits():
    # all functions are comparing credits in single table
    # an improvement would be applying accross tables using join()'s
    #e.g. All 30y bonds = all tables, BB credits, etc 
    
    def __init__(self,dbname):
        self.DB = PySQLite(dbname)
        self.TargetDict={'FiveISIN':'FiveTarget','SevenISIN':'SevenTarget','TenISIN':'TenTarget',
                    'ThirtyISIN':'ThirtyTarget','PerpISIN':'PerpTarget'}
        
#         self.dbname=dbname
#         conn = sqlite3.connect(self.dbname)
#         c = conn.cursor()
#         conn.close()

    def Current_Market_Data(self,security,field):
        con = pdblp.BCon(debug=False,port=8194,timeout=5000)
        con.start()

        # df=con.bdh(self.ticker+" equity",'px_last','20201211','20201214')
        # print(df.index)
#         cusip = self.benchmarkdict[bond]
        df = con.ref(security,field)
        return(df.iloc[0]['value'])
    
    def Historical_Market_Data(self,ticker,field,beg_date,end_date):
        con = pdblp.BCon(debug=False,port=8194,timeout=5000)
        con.start()
#         print(ticker,field,beg_date,end_date)

#         df=con.bdh(ticker,field,beg_date,end_date)
#         df2=df.iloc[1]+df.iloc[-1]

        df=con.ref_hist(ticker,field,[beg_date,end_date])
#         df = con.ref(ticker,field,'20200105','20200105')


        print(df)
        # print(df.index)
#         cusip = self.benchmarkdict[bond]
#         df = con.ref(ISIN,field)
#         return(df.iloc[0]['value'])
    

#         ET equity px_last 2020-02-01 2020-12-29
        
    def CreditTargets(self,table,security,field):
        
        #returns levels vs targets for a credits in a single table
        df = self.DB.DBdescription(table)
        df = df.iloc[:-1] #cutoff last row bc of placeholder         

        corplist=df[security].apply(lambda x: x+" corp")
    
        levels=corplist.apply(self.Current_Market_Data,args=[field])
        difference=levels-df[self.TargetDict[security]]
        
        comps=pd.concat([df['Name'],df[self.TargetDict[security]],levels,difference],axis=1,keys=['Name','Target','Actual','Up/(Down)'])
        comps_sorted=comps.sort_values(by=['Up/(Down)'],ascending=False)
        index=comps_sorted.index
        index.name=security
#         comps_sorted=comps_sorted.map('${:,.2f}'.format)
        
        
        return(comps_sorted)
    
    def EquityMoves(self,period,table):
        bbgcode=bbgdict[period]
        df = self.DB.DBdescription(table)
        df = df.iloc[:-1] #cutoff last row bc of placeholder
        equitylist=df['Ticker'].apply(lambda x: x+" equity")
        prices=equitylist.apply(self.Current_Market_Data,args=[bbgcode])
        listprices=pd.concat([df['Name'],prices],axis=1,keys=['Name',period]) 
        listprices_sorted=listprices.sort_values(by=[period],ascending=True)
        
        return(listprices_sorted)
        
    
    def BestCredit(self,df,field,limit):
        #returns highest/lowest value
        if limit=='min':
            target=df[field].idxmin()
        elif limit=='max':
            target=df[field].idxmax()           
        
        return(df.loc[target])
                    
            
            
def Main():    
    Comp=CompareCredits('test1228v1.db')
    ThirtyComps=Comp.CreditTargets('test_table1','ThirtyISIN','BLOOMBERG_MID_G_SPREAD')
#     FiveComps=Comp.CreditTargets('test_table1','FiveISIN','BLOOMBERG_MID_G_SPREAD')
#     test=ThirtyComps.sort_values(by=['Up/(Down)'])
#     print(test)
#     print(type(ThirtyComps.iloc[2]['Actual']))
#     print(ThirtyComps['Actual'] - ThirtyComps['Target'])
#     print(ThirtyComps)
#     print(FiveComps)
    Date='20200201'
    
    
    oneyr=Comp.EquityMoves('1yrchange','test_table1')
    onemo=Comp.EquityMoves('1mchange','test_table1')
#     print(oneyr)
    print(Comp.BestCredit(oneyr,'1yrchange','min'))
    
    
#     print(ThirtyComps)
    print(Comp.BestCredit(ThirtyComps,'Up/(Down)','max'))
    
#     print(onemo)
    
    
#     Comp.Historical_Market_Data('ET equity','px_last',Date,Today_f)
#     print(Comp.Current_Market_Data('ET equity','CURRENT_TRR_1YR'))
    
    
    
    


Main()
           
            

Name           PAA
1yrchange   -54.53
Name: 1, dtype: object
Name           WES
Target      320.00
Actual      418.27
Up/(Down)    98.27
Name: 3, dtype: object
